In [ ]:
!pip install mne

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import mne

torch.manual_seed(42)
np.random.seed(42)

In [ ]:
!pip install scikit-learn

# pre process the raw edf files

In [ ]:
def load_and_preprocess_edf(file_path, label, target_length=30000):
    raw = mne.io.read_raw_edf(file_path, preload=True)

    raw.filter(1, 50) # applying the lower limit and notch filter 

    data = raw.get_data() # data to numpy array

    data = (data - data.mean()) / data.std() # normalise length around the standard deviation

    current_length = data.shape[1]
    if current_length < target_length:
        pad_width = ((0, 0), (0, target_length - current_length))
        data = np.pad(data, pad_width, mode='constant')
    elif current_length > target_length:
        data = data[:, :target_length]

    return data, label


Defining the directory for classification

In [ ]:
  data_dir = "/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0"



class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = [torch.FloatTensor(x) for x in X]
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], torch.LongTensor([self.y[idx]])

# Update the load_dataset function
def load_dataset(data_dir, target_length=30000):
    X, y = [], []
    for filename in os.listdir(data_dir):
        if filename.endswith(".edf"):
            file_path = os.path.join(data_dir, filename)
            label = 0 if "rest" in filename.lower() else 1  # Assume filename contains "rest" for rest state
            data, label = load_and_preprocess_edf(file_path, label, target_length)
            X.append(data)
            y.append(label)
    return X, y

from sklearn.model_selection import train_test_split


Loading the dataset EDF format

In [ ]:
# Load the dataset with a fixed target_length
target_length = 30000  # You can adjust this value based on your data
X, y = load_dataset(data_dir, target_length)

# Create train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = EEGDataset(X_train, y_train)
test_dataset = EEGDataset(X_test, y_test)

Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/Subject25_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (3.302 s)

Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/Subject17_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info s

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (3.302 s)

Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/Subject28_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
--------------------

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/Subject35_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (3.302 s)

Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/Subject20_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info s

...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/Subject17_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/Subject29_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Defining batchsize and creating a dataset class and train test split

In [ ]:
class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.FloatTensor(self.X[idx]), torch.LongTensor([self.y[idx]])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = EEGDataset(X_train, y_train)
test_dataset = EEGDataset(X_test, y_test)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Defining the main CNN that is EEGNet with a forward pass

In [ ]:
class EEGNet(nn.Module):
    def __init__(self, num_channels, num_samples, num_classes):
        super(EEGNet, self).__init__()
        self.F1 = 8
        self.D = 2
        self.F2 = 16

        self.conv1 = nn.Conv2d(1, self.F1, (1, 64), padding=(0, 32), bias=False)
        self.batchnorm1 = nn.BatchNorm2d(self.F1)

        self.depthwise_conv = nn.Conv2d(self.F1, self.F1 * self.D, (num_channels, 1), groups=self.F1, bias=False)
        self.batchnorm2 = nn.BatchNorm2d(self.F1 * self.D)
        self.activation = nn.ELU()

        self.separable_conv = nn.Conv2d(self.F1 * self.D, self.F2, (1, 16), padding=(0, 8), bias=False)
        self.batchnorm3 = nn.BatchNorm2d(self.F2)

        self.avg_pool = nn.AvgPool2d((1, 4))

        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(self.F2 * (num_samples // 4), num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.depthwise_conv(x)
        x = self.batchnorm2(x)
        x = self.activation(x)
        x = self.separable_conv(x)
        x = self.batchnorm3(x)
        x = self.activation(x)
        x = self.avg_pool(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x

In [ ]:
# Update the model initialization
num_channels = X[0].shape[0]
num_samples = target_length
num_classes = 2
model = EEGNet(num_channels, num_samples, num_classes)

defining the training and evaluation function

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device).squeeze()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    accuracy = 100. * correct / total
    return running_loss / len(loader), accuracy

def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device).squeeze()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100. * correct / total
    return running_loss / len(loader), accuracy

Keeping lessser epochs because 4th iteration gives what we need 

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    print()

# Save the trained model
torch.save(model.state_dict(), "eegnet_model.pth")

Epoch 1/10
Train Loss: 0.3713, Train Acc: 71.93%
Test Loss: 0.1698, Test Acc: 100.00%

Epoch 2/10
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0114, Test Acc: 100.00%

Epoch 3/10
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0002, Test Acc: 100.00%

Epoch 4/10
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0000, Test Acc: 100.00%

Epoch 5/10
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0000, Test Acc: 100.00%

Epoch 6/10
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0000, Test Acc: 100.00%

Epoch 7/10
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0000, Test Acc: 100.00%

Epoch 8/10
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0000, Test Acc: 100.00%

Epoch 9/10
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0000, Test Acc: 100.00%

Epoch 10/10
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0000, Test Acc: 100.00%

